In [ ]:
20

In [ ]:
20

In [ ]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model




### Register model

In [ ]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [ ]:


from federatedscope.contrib.trainer.laplacian_trainer_with_domain_separation_with_summation_1MINE_VAE_separated import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

### Register metric

### Create the config

#### Set data, model, trainer and metric

In [ ]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("scripts/B-FHTL_exp_scripts/Graph-DT/fed_dom_sep_MI.yaml")
cfg.data.save_dir = 'test_dir'
# cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer'
cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']
cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance= 1e2
cfg.params.diff_importance = 0.1  # 1
cfg.params.mine_lr = 0.01
cfg.params.eps=1e-20
cfg.params.p=0.
cfg.params.lam = 0.
cfg.params.recon_importance = 0.1
cfg.params.kld_importance = 0.1


#### configure other options

In [ ]:
#cfg.use_gpu = True 0.05 [0.02-0.03] 0.04-0.06
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'

cfg.federate.method = \
    f'Laplacian_MINE_VAE_test_no_KLD'

#cfg.federate.local_update_steps = 20000000
cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 2000
cfg.federate.client_num = 2
cfg.early_stop.patience = 3000
#cfg.train.optimizer.lr = 0.001
#cfg.train.optimizer.weight_decay = 0.0005
#cfg.grad.grad_clip = 2.0
cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'


In [ ]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(F/home/michael/Desktop/backup files/FederatedScope/data/CIKM22Competitionalse)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [ ]:
from yacs.config import CfgNode
client_cfg_file = "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_theirs.yaml"# "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_ours_lr_local_steps.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [ ]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


In [ ]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_MI_client import LaplacianDomainSeparationMIClient
from federatedscope.contrib.workers.laplacian_server_dom_sep import LaplacianServerDomSep

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


In [17]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_1MINE_VAE_Separated_client import LaplacianDomainSeparation1MINE_Separated_Client
from federatedscope.contrib.workers.laplacian_server import LaplacianServer

Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianDomainSeparation1MINE_Separated_Client,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

mi_local_global: 0.17895346879959106
mi_global_fixed: -1.519773244857788
rec_loss: 0.1676005721092224
kld_loss: 0.5204925537109375
mi_local_global: 0.26366668939590454
mi_global_fixed: -1.647637128829956
rec_loss: 0.14964152872562408
kld_loss: 0.5297309756278992


2023-01-11 19:19:14,448 (laplacian_with_domain_separation_1MINE_VAE_Separated_client:144)INFO: {'Role': 'Client #2', 'Round': 202, 'Results_raw': {'train_loss': 28.915263, 'train_total': 512, 'train_avg_loss': 0.056475, 'train_imp_ratio': -113.11368}}
2023-01-11 19:19:14,450 (server:480)INFO: {'Role': 'Server #', 'Round': 202, 'Results_avg': {'test_loss': 65.486218, 'test_total': 88.5, 'test_avg_loss': 0.589868, 'test_imp_ratio': -51.252937, 'val_loss': 50.997436, 'val_total': 88.0, 'val_avg_loss': 0.462727, 'val_imp_ratio': -44.678816}}
2023-01-11 19:19:14,451 (monitor:513)INFO: current_best=-10.402743, should_save=False
2023-01-11 19:19:14,452 (monitor:513)INFO: current_best=-10.402743, should_save=False
2023-01-11 19:19:14,523 (laplacian_server:160)INFO: Server #0: Starting evaluation at the end of round 202.
2023-01-11 19:19:14,526 (laplacian_server:167)INFO: ----------- Starting a new training round (Round #203) -------------
2023-01-11 19:19:14,686 (client:410)INFO: {'Role': 'Cli

in val or test
mi_local_global: -0.539800226688385
mi_global_fixed: -3.2164597511291504
rec_loss: 0.7771382331848145
kld_loss: 1.4875636100769043
mi_local_global: -0.6111831068992615
mi_global_fixed: -2.998499631881714
rec_loss: 0.7755870223045349
kld_loss: 1.5264103412628174
in val or test
mi_local_global: -0.5666018724441528
mi_global_fixed: -3.170928955078125
rec_loss: 0.7575591802597046
kld_loss: 1.51613450050354
mi_local_global: -0.5898518562316895
mi_global_fixed: -2.973360061645508
rec_loss: 0.7455371618270874
kld_loss: 1.5043823719024658
in val or test
mi_local_global: 0.280279278755188
mi_global_fixed: -2.659762382507324
rec_loss: 0.08040502667427063
kld_loss: 0.548511266708374


2023-01-11 19:19:14,822 (client:410)INFO: {'Role': 'Client #2', 'Round': 203, 'Results_raw': {'test_loss': 2.415355, 'test_total': 64, 'test_avg_loss': 0.03774, 'test_imp_ratio': -42.414798, 'val_loss': 2.461889, 'val_total': 63, 'val_avg_loss': 0.039078, 'val_imp_ratio': -47.462622}}
2023-01-11 19:19:14,823 (monitor:513)INFO: current_best=-37.875595, should_save=False


in val or test
mi_local_global: 0.30433857440948486
mi_global_fixed: -3.0243730545043945
rec_loss: 0.11812476813793182
kld_loss: 0.5124056935310364
mi_local_global: 0.4424206018447876
mi_global_fixed: -1.9663292169570923
rec_loss: 0.14224311709403992
kld_loss: 0.5210649967193604
mi_local_global: 0.4552731513977051
mi_global_fixed: -2.0968666076660156
rec_loss: 0.16045016050338745
kld_loss: 0.528276801109314
mi_local_global: 0.4157118797302246
mi_global_fixed: -1.8770787715911865
rec_loss: 0.1763809472322464
kld_loss: 0.4993744194507599
mi_local_global: 0.4074058532714844
mi_global_fixed: -1.7687997817993164
rec_loss: 0.16837552189826965
kld_loss: 0.520714282989502
mi_local_global: 0.23122698068618774
mi_global_fixed: -1.4051756858825684
rec_loss: 0.17202983796596527
kld_loss: 0.5437097549438477
mi_local_global: 0.2413923740386963
mi_global_fixed: -1.7943530082702637
rec_loss: 0.18520185351371765
kld_loss: 0.5175331234931946
mi_local_global: 0.08363896608352661
mi_global_fixed: -1.92634

...

2023-01-11 19:21:42,479 (client:410)INFO: {'Role': 'Client #2', 'Round': 232, 'Results_raw': {'test_loss': 2.197988, 'test_total': 64, 'test_avg_loss': 0.034344, 'test_imp_ratio': -29.598351, 'val_loss': 3.008898, 'val_total': 63, 'val_avg_loss': 0.04776, 'val_imp_ratio': -80.227464}}
2023-01-11 19:21:42,481 (monitor:513)INFO: current_best=-37.875595, should_save=False


mi_local_global: 0.3405361771583557
mi_global_fixed: -3.1465773582458496
rec_loss: 0.11045676469802856
kld_loss: 0.46771758794784546
in train
mi_local_global: 0.34584754705429077
mi_global_fixed: -2.167393922805786
rec_loss: 0.1771874576807022
kld_loss: 0.47677022218704224
mi_local_global: 0.5118789672851562
mi_global_fixed: -1.7387897968292236
rec_loss: 0.1620887666940689
kld_loss: 0.47112053632736206
mi_local_global: 0.40026944875717163
mi_global_fixed: -1.7552130222320557
rec_loss: 0.14808107912540436
kld_loss: 0.4758768677711487
mi_local_global: 0.6279841065406799
mi_global_fixed: -1.4607298374176025
rec_loss: 0.1550244688987732
kld_loss: 0.4747731685638428
mi_local_global: 0.18470674753189087
mi_global_fixed: -1.4741370677947998
rec_loss: 0.1583646535873413
kld_loss: 0.4873645603656769
mi_local_global: 0.10003036260604858
mi_global_fixed: -1.4122141599655151
rec_loss: 0.15058930218219757
kld_loss: 0.4853745698928833
mi_local_global: 0.0473136305809021
mi_global_fixed: -1.544793605

2023-01-11 19:21:44,099 (laplacian_with_domain_separation_1MINE_VAE_Separated_client:144)INFO: {'Role': 'Client #2', 'Round': 232, 'Results_raw': {'train_loss': 39.014445, 'train_total': 512, 'train_avg_loss': 0.0762, 'train_imp_ratio': -187.547539}}


in train
mi_local_global: -0.6024003028869629
mi_global_fixed: -1.9342536926269531
rec_loss: 0.7718844413757324
kld_loss: 1.4816261529922485
mi_local_global: -0.5019956231117249
mi_global_fixed: -1.843017816543579
rec_loss: 0.7622277140617371
kld_loss: 1.4989142417907715
mi_local_global: -0.6056339144706726
mi_global_fixed: -1.8752388954162598
rec_loss: 0.764568030834198
kld_loss: 1.5089466571807861
mi_local_global: -0.5012573003768921
mi_global_fixed: -1.6665534973144531
rec_loss: 0.7705269455909729
kld_loss: 1.4455221891403198
mi_local_global: -0.5204504728317261
mi_global_fixed: -1.8601081371307373
rec_loss: 0.7690281867980957
kld_loss: 1.4658453464508057
mi_local_global: -0.5978037118911743
mi_global_fixed: -2.054941177368164
rec_loss: 0.762418270111084
kld_loss: 1.468303918838501
mi_local_global: -0.501045286655426
mi_global_fixed: -1.544421672821045
rec_loss: 0.7645395994186401
kld_loss: 1.4531888961791992
mi_local_global: -0.562314510345459
mi_global_fixed: -2.2275631427764893
r

KeyboardInterrupt: 

client 0 times: 0.346s
bei normal fedavg: 0.15s
mit MINE: 1.7 Sekunden


+

In [ ]:
777-431


In [ ]:
data

In [ ]:
55: -41 with 0 KLD
mit 0.1 KLD: -31

In [ ]:
import os

,,0After 10 rounds:

diff: 0, csd: 0:
server: -8
c1 t/val: 0.375 /0.37
c2 t/val: 0.678 /0.606
c3 t/val: 0.699 /0.608
c4 t/val: 0.599 /0.537

diff: 0, csd: 0:
server: -4.5
c1 t/val: 0.372879 /0.328
c2 t/val: 0.678992 /0.605809
c3 t/val: 0.70166 /0.608609
c4 t/val: 0.597455 /0.534253



diff: 10, csd: 0:
server: -35
c1 t/val: 0.482/0.52
c2 t/val: 0.694/0.65
c3 t/val: 0.724/0.65
c4 t/val: 0.674/0.575

diff: 1, csd: 0:
server: -9.6
c1 t/val: 0.378194/0.319349
c2 t/val: 0.67461/0.609163
c3 t/val: 0.705559/0.615019
c4 t/val: 0.605581/0.572086

diff: 0, csd: 1e4:
server: -18
c1 t/val: 0.280962/0.23074
c2 t/val: 0.682652/0.603175
c3 t/val: 0.699094/0.67027
c4 t/val: 0.594985/0.609611

diff: 0 csd: 1e8
server: -18
c1 t/val: 0.2606/0.348(0.197)
c2 t/val: 0.653/0.0.602
c3 t/val: 0.691/0.597
c4 t/val: 0.581/0.629

diff: 0 csd: 1e8
server: -12.7
c1 t/val: 0.262647/0.249875
c2 t/val: 0.649273/0.0.600435
c3 t/val: 0.696132/0.615158
c4 t/val: 0.586597/0.656992


diff: 0.1 csd: 0
server: -11
c1 t/val: 0.2606/0.332837
c2 t/val: 0.678326/0.0.610099
c3 t/val: 0.704083/0.616179
c4 t/val: 0.604177/0.559658




In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']